In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import numpy as np

embeddings_names = [
    # "blood_cell_scrna",
    # "lymphoma",
    "cifar_100",
    "mnist",
]
sigs = [
    # [(1, 2), (0, 2), (-1, 2), (-1, 2), (-1, 2)],
    # [(1, 2), (1, 2)],
    [(1, 2), (1, 2), (1, 2), (1, 2)],
    [(1, 2), (0, 2), (-1, 2)],
]
n_trials = 10
sets = ["train", "test"]
datasets = ["X", "y"]

bad = []
for embedding in embeddings_names:
    for trial in range(n_trials):
        for set_name in sets:
            for dataset in datasets:
                my_data = np.load(f"../data/{embedding}/embeddings/{dataset}_{set_name}_{trial}.npy")
                if np.isnan(my_data).any():
                    bad.append((embedding, trial, set_name, dataset))
                    print(embedding, trial, set_name, dataset)
                print(my_data.shape)
print(bad)

(40000, 12)
(40000,)
(10000, 12)
(10000,)
(40000, 12)
(40000,)
(10000, 12)
(10000,)
(40000, 12)
(40000,)
(10000, 12)
(10000,)
(40000, 12)
(40000,)
(10000, 12)
(10000,)
(40000, 12)
(40000,)
(10000, 12)
(10000,)
(40000, 12)
(40000,)
(10000, 12)
(10000,)
(40000, 12)
(40000,)
(10000, 12)
(10000,)
(40000, 12)
(40000,)
(10000, 12)
(10000,)
(40000, 12)
(40000,)
(10000, 12)
(10000,)
(40000, 12)
(40000,)
(10000, 12)
(10000,)
(48000, 8)
(48000,)
(12000, 8)
(12000,)
(48000, 8)
(48000,)
(12000, 8)
(12000,)
(48000, 8)
(48000,)
(12000, 8)
(12000,)
(48000, 8)
(48000,)
(12000, 8)
(12000,)
(48000, 8)
(48000,)
(12000, 8)
(12000,)
(48000, 8)
(48000,)
(12000, 8)
(12000,)
(48000, 8)
(48000,)
(12000, 8)
(12000,)
(48000, 8)
(48000,)
(12000, 8)
(12000,)
(48000, 8)
(48000,)
(12000, 8)
(12000,)
(48000, 8)
(48000,)
(12000, 8)
(12000,)
[]


In [5]:
import embedders
import pandas as pd
from tqdm.notebook import tqdm

N_SAMPLES = 10_000

results = []
my_tqdm = tqdm(total=len(embeddings_names) * n_trials)
for embedding, sig in zip(embeddings_names, sigs):
    pm = embedders.manifolds.ProductManifold(signature=sig)
    for trial in range(n_trials):
        X_train = np.load(f"../data/{embedding}/embeddings/X_train_{trial}.npy")
        y_train = np.load(f"../data/{embedding}/embeddings/y_train_{trial}.npy")
        X_test = np.load(f"../data/{embedding}/embeddings/X_test_{trial}.npy")
        y_test = np.load(f"../data/{embedding}/embeddings/y_test_{trial}.npy")

        # Randomly subsample
        if len(X_train) > N_SAMPLES:
            idx = np.random.choice(X_train.shape[0], N_SAMPLES, replace=False)
            X_train = X_train[idx]
            y_train = y_train[idx]

        if len(X_test) > N_SAMPLES:
            idx = np.random.choice(X_test.shape[0], N_SAMPLES, replace=False)
            X_test = X_test[idx]
            y_test = y_test[idx]

        res = embedders.benchmarks.benchmark(
            X=None,
            y=None,
            X_train=X_train,
            X_test=X_test,
            y_train=y_train,
            y_test=y_test,
            pm=pm,
            # models=["sklearn_dt", "product_dt"],
            max_depth=3,
            # batch_size=1,
        )
        res["embedding"] = embedding
        res["trial"] = trial

        results.append(res)
        my_tqdm.update(1)

results = pd.DataFrame(results)

  0%|          | 0/20 [00:00<?, ?it/s]

In [6]:
results.to_csv("../data/results/vae.tsv", sep="\t", index=False)